<a href="https://colab.research.google.com/github/danisilva1110-spec/PFPython/blob/main/robot_dynamics_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook de Cinemática e Dinâmica Simbólica em Python (Colab)Este notebook replica o fluxo do projeto original em MATLAB usando Python puro (SymPy) com aceleração opcional via JAX/TPU/GPU no Google Colab. Ele calcula, de forma simbólica, a cinemática direta e as equações de movimento (M, C, G) de um robô em cadeia aberta com quaisquer parâmetros DH e propriedades inerciais fornecidos.**Destaques**- Construção simbólica com `sympy` (portável e rápida).- Geração de funções numéricas (`numpy`, `jax`) para execução acelerada em GPU/TPU no Colab.- Estrutura genérica: suporte a juntas revolutas e prismáticas, posições de centro de massa arbitrárias e tensores de inércia 3x3.> Basta preencher a célula de definição do robô com seus parâmetros (DH, massas, centros de massa e inércia) — inclusive os modelos ABB/SCARA do repositório — e executar as células na ordem.

## 1. AmbienteInstala SymPy e JAX (para aceleração opcional em GPU/TPU). Em Colab, o JAX já vem preparado para TPU; para GPU use o backend CUDA que o Colab disponibiliza.

In [1]:
!pip -q install sympy jax jaxlib > /dev/null

## 2. Importações e utilidades

In [2]:
import sympy as spfrom sympy import Matrixfrom dataclasses import dataclassfrom typing import List, Literal, Tuplesp.init_printing(use_latex='mathjax')

SyntaxError: invalid syntax (ipython-input-1206467819.py, line 1)

## 3. Estruturas de dados do robô- `Joint`: tipo (revoluta ou prismática) e índice do parâmetro variável.- `Link`: parâmetros DH modificados, massa, centro de massa (em coordenadas do elo) e tensor de inércia (no frame do elo).- `RobotModel`: contém a lista de elos e a gravidade.

In [ ]:
JointType = Literal['R', 'P']@dataclassclass Joint:    joint_type: JointType    theta: sp.Symbol    d: sp.Symbol    a: sp.Symbol    alpha: sp.Symbol@dataclassclass Link:    joint: Joint    mass: sp.Symbol    com: Matrix  # (3, 1)    inertia: Matrix  # (3, 3) no frame do elo@dataclassclass RobotModel:    links: List[Link]    gravity: Matrix    @property    def dof(self) -> int:        return len(self.links)

## 4. Funções auxiliaresInclui geração da matriz de transformação homogênea, jacobianos de posição/rotação e composição das transformações ao longo da cadeia.

In [ ]:
def dh_transform(theta, d, a, alpha) -> Matrix:    ct, st = sp.cos(theta), sp.sin(theta)    ca, sa = sp.cos(alpha), sp.sin(alpha)    return Matrix([        [ct, -st * ca,  st * sa, a * ct],        [st,  ct * ca, -ct * sa, a * st],        [0,        sa,        ca,      d],        [0,         0,         0,      1],    ])def forward_kinematics(model: RobotModel) -> Tuple[List[Matrix], List[Matrix]]:    Ts, origins = [], [Matrix([0, 0, 0])]    T = sp.eye(4)    for link in model.links:        T = T * dh_transform(link.joint.theta, link.joint.d, link.joint.a, link.joint.alpha)        Ts.append(T)        origins.append(T[:3, 3])    return Ts, originsdef spatial_jacobians(model: RobotModel, origins: List[Matrix]) -> Tuple[List[Matrix], List[Matrix]]:    z_axes = [Matrix([0, 0, 1])]    R = sp.eye(3)    for T in forward_kinematics(model)[0]:        R = T[:3, :3]        z_axes.append(R[:, 2])    Jvs, Jws = [], []    Ts, _ = forward_kinematics(model)    for i, link in enumerate(model.links):        o_i = origins[i]        o_com = origins[i + 1] + Ts[i][:3, :3] * link.com        Jv_cols, Jw_cols = [], []        for j in range(model.dof):            z = z_axes[j]            o_j = origins[j]            if model.links[j].joint.joint_type == 'R':                Jv_cols.append(z.cross(o_com - o_j))                Jw_cols.append(z)            else:  # prismática                Jv_cols.append(z)                Jw_cols.append(Matrix([0, 0, 0]))        Jvs.append(Matrix.hstack(*Jv_cols))        Jws.append(Matrix.hstack(*Jw_cols))    return Jvs, Jws

## 5. Energia, matrizes M/C/G e equações de movimentoA seguir calculamos a energia cinética/potencial de cada elo e aplicamos Lagrange para obter `M(q)`, `C(q, q̇)` e `G(q)`, bem como o vetor de torques/forças `\tau`.

In [ ]:
def dynamics(model: RobotModel, qs: List[sp.Symbol], dqs: List[sp.Symbol], ddqs: List[sp.Symbol]):    Ts, origins = forward_kinematics(model)    Jvs, Jws = spatial_jacobians(model, origins)    K, U = 0, 0    for i, link in enumerate(model.links):        R = Ts[i][:3, :3]        v = Jvs[i] * Matrix(dqs)        w = Jws[i] * Matrix(dqs)        I_world = R * link.inertia * R.T        K += sp.simplify(0.5 * link.mass * (v.T * v)[0] + 0.5 * (w.T * I_world * w)[0])        U += sp.simplify(link.mass * model.gravity.dot(origins[i + 1] + R * link.com))    L = K - U    tau = []    for q, dq, ddq in zip(qs, dqs, ddqs):        dL_dq = sp.diff(L, q)        dL_ddq = sp.diff(L, dq)        d_dt_dL_ddq = sp.diff(dL_ddq, *(list(zip(qs, dqs))))        tau.append(sp.simplify(d_dt_dL_ddq - dL_dq))    tau = Matrix(tau)    M = tau.jacobian(ddqs)    Cg = tau.subs({dd: 0 for dd in ddqs})    C = Cg - Cg.subs({dq: 0 for dq in dqs})    G = Cg.subs({dq: 0 for dq in dqs})    return sp.simplify(M), sp.simplify(C), sp.simplify(G), sp.simplify(tau)

## 6. Exemplo: manipulador de 3 DOF (revoluto)Preencha a célula abaixo com os parâmetros do seu robô. Para usar os dados do ABB ou SCARA, basta adaptar a lista `links` com os DH, massas, centros de massa e inércias exportados do MATLAB.

In [ ]:
# Variáveis generalizadasq1, q2, q3 = sp.symbols('q1 q2 q3')dq1, dq2, dq3 = sp.symbols('dq1 dq2 dq3')ddq1, ddq2, ddq3 = sp.symbols('ddq1 ddq2 ddq3')# Parâmetros DH (theta, d, a, alpha) com juntas revolutaslink1 = Link(Joint('R', q1, sp.Symbol('d1'), sp.Symbol('a1'), sp.Symbol('alpha1')),             mass=sp.Symbol('m1'), com=Matrix([0, 0, sp.Symbol('lz1')]),             inertia=sp.diag(*sp.symbols('Ixx1 Iyy1 Izz1')))link2 = Link(Joint('R', q2, sp.Symbol('d2'), sp.Symbol('a2'), sp.Symbol('alpha2')),             mass=sp.Symbol('m2'), com=Matrix([sp.Symbol('lx2'), 0, 0]),             inertia=sp.diag(*sp.symbols('Ixx2 Iyy2 Izz2')))link3 = Link(Joint('R', q3, sp.Symbol('d3'), sp.Symbol('a3'), sp.Symbol('alpha3')),             mass=sp.Symbol('m3'), com=Matrix([sp.Symbol('lx3'), 0, 0]),             inertia=sp.diag(*sp.symbols('Ixx3 Iyy3 Izz3')))robot = RobotModel(links=[link1, link2, link3], gravity=Matrix([0, 0, -sp.Symbol('g')]))M, C, G, tau = dynamics(robot, [q1, q2, q3], [dq1, dq2, dq3], [ddq1, ddq2, ddq3])print('Matriz de inércia M(q):')sp.pprint(M)print('\nMatriz de Coriolis/Centrífuga C(q,q̇):')sp.pprint(C)print('\nVetor gravitacional G(q):')sp.pprint(G)print('\nEquações de movimento τ = M·q̈ + C + G:')sp.pprint(tau)

## 7. Funções aceleradas (GPU/TPU com JAX)Use `sympy.lambdify` para gerar funções numéricas vetorizadas. No Colab, defina `backend='jax'` para executar em GPU/TPU sem mudar o restante do código.

In [ ]:
import numpy as npimport jax.numpy as jnpfrom sympy.utilities.lambdify import lambdify# Exemplo: função para M(q) no backend JAXparams = (sp.symbols('a1 a2 a3 d1 d2 d3 alpha1 alpha2 alpha3'),          sp.symbols('m1 m2 m3'),          sp.symbols('lx2 lx3 lz1'),          sp.symbols('Ixx1 Iyy1 Izz1 Ixx2 Iyy2 Izz2 Ixx3 Iyy3 Izz3'),          sp.symbols('g'))flat_symbols = list(params[0] + params[1] + params[2] + params[3] + (params[4],)) + [q1, q2, q3]M_func = lambdify(flat_symbols, M, modules='jax')# Avaliação numérica vetorizada (exemplo de chamada)def inertia_numeric(values: dict):    args = [values[s] for s in flat_symbols]    return jnp.array(M_func(*args))